# Load libraries

In [42]:
%matplotlib inline

import os
import math

import configparser

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import sys
sys.path.append('../..')

from scripps.utils import read_datasets, norm1d, bayesian_linear_regression
from IPython.core.display import clear_output

In [43]:
#np.random.seed(23) #Not working

# Load all mark2cure citizen scientist annotations

In [44]:
CF = read_datasets.get_configuration()

def read_disease_annotations():
    annotations = pd.DataFrame()
    anno = read_datasets.load_dataset('m2c_citizen_disease', 'files1')
    anno[3].replace(['I-Disease', 'O'], [1, 0], inplace=True)
    annotations = pd.concat([annotations, anno[0].rename('Token')], axis=1)
    annotations = pd.concat([annotations, anno[3].rename('Annotator1')], axis=1)
    for i in range(2, 6):
        anno = read_datasets.load_dataset('m2c_citizen_disease', 'files{}'.format(i))
        anno[3].replace(['I-Disease', 'O'], [1, 0], inplace=True)
        annotations = pd.concat([annotations, anno[3].rename('Annotator{}'.format(i))], axis=1)
    return annotations

def read_phenotype_annotations():
    annotations = pd.DataFrame()
    anno = read_datasets.load_dataset('m2c_citizen_phenotype', 'files1')
    anno[3].replace(['I-Phenotype', 'O'], [1, 0], inplace=True)
    annotations = pd.concat([annotations, anno[0].rename('Token')], axis=1)
    annotations = pd.concat([annotations, anno[3].rename('Annotator1')], axis=1)
    for i in range(2, 6):
        anno = read_datasets.load_dataset('m2c_citizen_phenotype', 'files{}'.format(i))
        anno[3].replace(['I-Phenotype', 'O'], [1, 0], inplace=True)
        annotations = pd.concat([annotations, anno[3].rename('Annotator{}'.format(i))], axis=1)
    return annotations

In [45]:
disease_annotations = read_disease_annotations()
phenotype_annotations = read_phenotype_annotations()

In [46]:
disease_annotations

,Token,Annotator1,Annotator2,Annotator3,Annotator4,Annotator5
0,Haematuria,0,0,1,0,1
1,and,0,0,0,0,0
2,abdominal,1,1,1,1,1
3,aortic,1,1,1,1,1
4,aneurysm,1,1,1,1,1
5,.,0,0,0,0,0
6,Haematuria,0,0,1,0,1
7,and,0,0,0,0,0
8,left,0,0,0,0,0
9,loin,0,0,0,0,0


In [47]:
phenotype_annotations

,Token,Annotator1,Annotator2,Annotator3,Annotator4,Annotator5
0,Haematuria,1,1,1,1,1
1,and,0,0,0,0,0
2,abdominal,0,0,0,1,0
3,aortic,0,0,0,1,0
4,aneurysm,0,0,0,1,0
5,.,0,0,0,0,0
6,Haematuria,1,1,1,1,1
7,and,0,0,0,0,0
8,left,1,1,0,0,1
9,loin,1,1,0,1,1


In [48]:
num_annotators = 5
num_tokens = disease_annotations.shape[0]

disease_relevant = pd.DataFrame()
phenotype_relevant = pd.DataFrame()

relevant = []
for k in range(num_tokens):
    disease = 0
    phenotype = 0
    
    for j in range(1, num_annotators+1):                
        d = disease_annotations['Annotator{}'.format(j)][k]
        p = phenotype_annotations['Annotator{}'.format(j)][k]
                
        disease += d 
        phenotype += p
        
    if disease == 0 and phenotype == 0:
        continue
    else:
        relevant.append(k)

relevant = np.array(relevant)
print '{} relevant words.'.format(relevant.size)
disease_relevant = disease_annotations.iloc[relevant]
phenotype_relevant = phenotype_annotations.iloc[relevant]


1611 relevant words.


# Initialize latent variables

In [49]:
K = disease_relevant.shape[0]
J = disease_relevant.shape[1] - 1

latent = {}

latent['theta'] = np.zeros(K)
latent['d_alpha'] = np.zeros(J)
latent['d_beta'] = np.zeros(J)
latent['d_z'] = np.zeros((K, J))

latent['theta'] = np.zeros(K)
latent['p_alpha'] = np.zeros(J)
latent['p_beta'] = np.zeros(J)
latent['p_z'] = np.zeros((K, J))

#Should these be separate for disease and phenotype?
latent['b0'] = np.zeros(2)
latent['B0_scale'] = 1
latent['m0'] = np.zeros(K) #TODO: ask HS regarding changing certain values here.
latent['C0_scale'] = 1

latent['K'] = K
latent['J'] = J

# Define functions for resampling using Gibbs sampling

In [50]:
def __get_alpha_beta_z(mode):
    if mode == 'disease':
        alpha = latent['d_alpha']
        beta = latent['d_beta']
        z = latent['d_z']
    else:
        alpha = latent['p_alpha']
        beta = latent['p_beta']
        z = latent['p_z']
    return alpha, beta, z
    

In [51]:
def __resample_z(alpha, beta, z, annotations):
    interval = [-100, 100] #Should this be changed?
    for k in range(K):
        for j in range(J):
            mean = alpha[j] + beta[j]*latent['theta'][k]
            std_dev = 1
            if(annotations['Annotator{}'.format(j+1)][relevant[k]] == 0):
                z[k, j] = norm1d.truncnormal(mean, std_dev, interval[0], 0)
            elif(annotations['Annotator{}'.format(j+1)][relevant[k]] == 1):
                z[k, j] = norm1d.truncnormal(mean, std_dev, 0, interval[1])
    
def resample_z(mode):
    alpha, beta, z = __get_alpha_beta_z(mode)
    if mode == 'disease':
        annotations = disease_relevant
    else:
        annotations = phenotype_relevant
    __resample_z(alpha, beta, z, annotations)
    return latent['d_z'] if mode == 'disease' else latent['p_z']

In [52]:
def __resample_alpha_beta(alpha, beta, z):
    for j in range(J):
        Y = z[:, j]
        X = np.vstack((np.ones(K), latent['theta'])).T
        
        WN, VN = bayesian_linear_regression.linreg_post(X, Y, latent['b0'], latent['B0_scale'], 1)
        sample = np.random.multivariate_normal(WN, VN)
        alpha[j] = sample[0]
        beta[j] = sample[1]

def resample_alpha_beta(mode):
    alpha, beta, z = __get_alpha_beta_z(mode)
    __resample_alpha_beta(alpha, beta, z)
    return alpha, beta

In [53]:
def resample_theta():
    for k in range(K):                
        Y = np.concatenate((latent['d_z'][k, :] - latent['d_alpha'], latent['p_z'][k, :] - latent['p_alpha']))
        X = np.concatenate((latent['d_beta'], latent['p_beta']))
        X = np.reshape(X, (-1, 1))
        WN, VN = bayesian_linear_regression.linreg_post(X, Y, latent['m0'][k], latent['C0_scale'], 1)
        latent['theta'][k] = np.random.normal(WN, math.sqrt(VN))    

In [54]:
def save_latent_variables():
    #TODO: save to csv.
    pass


def resample():
    d_z = resample_z('disease')
    p_z = resample_z('phenotype')
    
    resample_theta()
        
    d_alpha, d_beta = resample_alpha_beta('disease')
    p_alpha, p_beta = resample_alpha_beta('phenotype')    
    
    save_latent_variables()
    
    return latent['theta'], d_alpha, d_beta, p_alpha, p_beta, d_z, p_z
    
def has_converged():
    #TODO
    pass

def summarize_param(params, burn_in):
    return np.mean(params[burn_in:], axis=0)

def plot_param(params, param_name, burn_in=0):
    plt.hist(params)
    plt.xlabel(param_name)
    plt.show()

def run_sampling(num_iter, burn_in): 
    zs = []
    thetas = []
    d_alphas = []
    d_betas = []
    p_alphas = []
    p_betas = []
    d_zs = []
    p_zs = []
    
    #TODO: consider first step, when we decide if word is relevant or not.
    for it in xrange(1, num_iter+1):
        theta, d_alpha, d_beta, p_alpha, p_beta, d_z, p_z = resample()
        thetas.append(np.array(theta))
        d_alphas.append(np.array(d_alpha))
        d_betas.append(np.array(d_beta))
        p_alphas.append(np.array(p_alpha))
        p_betas.append(np.array(p_beta))
        d_zs.append(np.array(d_z))
        p_zs.append(np.array(p_z))
        
        if it%5 == 0:
            clear_output()
            print 'Iter {}/{} done.'.format(it, num_iter)
        if has_converged():
            break
    
    all_vars = {}            
    all_vars['thetas'] = thetas
    all_vars['d_alphas'] = d_alphas
    all_vars['d_betas'] = d_betas
    all_vars['p_alphas'] = p_alphas
    all_vars['p_betas'] = p_betas
    all_vars['d_zs'] = d_zs
    all_vars['p_zs'] = p_zs
    
    summary = {}                  
    summary['avg_d_alpha'] = summarize_param(d_alphas, burn_in)
    summary['avg_d_beta'] = summarize_param(d_betas, burn_in)
    summary['avg_p_alpha'] = summarize_param(p_alphas, burn_in)
    summary['avg_p_beta'] = summarize_param(p_betas, burn_in)
    summary['avg_theta'] = summarize_param(thetas, burn_in)    
    summary['avg_d_z'] = summarize_param(d_zs, burn_in)    
    summary['avg_p_z'] = summarize_param(p_zs, burn_in)    
      
    return all_vars, summary

In [55]:
all_vars, summary = run_sampling(2000, 200)

Iter 2000/2000 done.


In [56]:
print 'avg_d_alpha:', summary['avg_d_alpha']
print 'avg_d_beta:', summary['avg_d_beta']
print 'avg_p_alpha:', summary['avg_p_alpha']
print 'avg_p_beta:', summary['avg_p_beta']
print 'avg_theta:', summary['avg_theta']

avg_d_alpha: [-0.67146413 -0.67681871 -1.06182618 -1.19300591  0.10240882]
avg_d_beta: [2.20155178 1.97518912 2.15753036 2.04309986 2.55370613]
avg_p_alpha: [-0.96476047 -0.9357005  -0.83247455 -0.50760118 -0.53488689]
avg_p_beta: [ 0.10641583  0.1077217   0.26836786  0.38055351 -2.90351497]
avg_theta: [0.03713802 1.46893786 1.39637738 ... 1.30164185 1.36296288 1.25060409]


In [57]:
def get_annotator_bias(alpha, beta):
    # compute xr and xa using above latent vars.
#     xa = (4*alpha - beta*beta)/(4*beta)
#     xr = (4*alpha + beta*beta)/(4*beta)
    xa = (-2*alpha/beta - beta/2)/2
    xr = -2*alpha/beta - xa
    return xa, xr

def get_word_positions(theta, relevant):
    # convert theta locations to an interval [-1, 1]?    
#     x_coords = theta
#     y_coords = np.zeros(theta.size)

#     for i,token in enumerate(tokens):
#         x = x_coords[i]
#         y = y_coords[i]
#         plt.scatter(x, y, marker='x', color='red')
#         plt.text(x+0.3, y+0.3, token, fontsize=9)
#     plt.show()
    positions = pd.DataFrame()
    positions['Tokens'] = disease_annotations['Token']
    positions['Position'] = ''
    positions['Position'].iloc[relevant] = theta    
    for i in range(1, J+1):
        positions['D-A{}'.format(i)] = disease_annotations['Annotator{}'.format(i)]
        positions['P-A{}'.format(i)] = phenotype_annotations['Annotator{}'.format(i)]
    
    positions['Disease'] = np.sum([positions['D-A{}'.format(i)] for i in range(1, J+1)], axis=0)
    positions['Phenotype'] = np.sum([positions['P-A{}'.format(i)] for i in range(1, J+1)], axis=0)
    return positions

def plot_bias():
    # show the annotator's location along with the word's true position.
    pass


In [58]:
def alignment_to_expert_positions():
    pass

In [59]:
xpa, xpr = get_annotator_bias(summary['avg_p_alpha'], summary['avg_p_beta'])

In [60]:
xda, xdr = get_annotator_bias(summary['avg_d_alpha'], summary['avg_d_beta'])

In [61]:
xpa - xpr

array([-0.05320792, -0.05386085, -0.13418393, -0.19027675,  1.45175748])

In [62]:
xda - xdr

array([-1.10077589, -0.98759456, -1.07876518, -1.02154993, -1.27685306])

In [63]:
(xpa + xpr)/2

array([ 9.06594852,  8.68627679,  3.10199048,  1.33384969, -0.18422047])

In [64]:
(xda + xdr)/2

array([ 0.30499584,  0.3426602 ,  0.4921489 ,  0.58391953, -0.04010204])

In [65]:
print min(summary['avg_theta'])
print max(summary['avg_theta'])

-1.3396302873246326
1.7538437373832827


In [66]:
positions = get_word_positions(summary['avg_theta'], relevant)

In [67]:
positions

,Tokens,Position,D-A1,P-A1,D-A2,P-A2,D-A3,P-A3,D-A4,P-A4,D-A5,P-A5,Disease,Phenotype
0,Haematuria,0.037138,0,1,0,1,1,1,0,1,1,1,2,5
1,and,,0,0,0,0,0,0,0,0,0,0,0,0
2,abdominal,1.46894,1,0,1,0,1,0,1,1,1,0,5,1
3,aortic,1.39638,1,0,1,0,1,0,1,1,1,0,5,1
4,aneurysm,1.31554,1,0,1,0,1,0,1,1,1,0,5,1
5,.,,0,0,0,0,0,0,0,0,0,0,0,0
6,Haematuria,0.0365789,0,1,0,1,1,1,0,1,1,1,2,5
7,and,,0,0,0,0,0,0,0,0,0,0,0,0
8,left,-0.909746,0,1,0,1,0,0,0,0,0,1,0,3
9,loin,-0.767129,0,1,0,1,0,0,0,1,0,1,0,4


In [68]:
positions.iloc[relevant]

,Tokens,Position,D-A1,P-A1,D-A2,P-A2,D-A3,P-A3,D-A4,P-A4,D-A5,P-A5,Disease,Phenotype
0,Haematuria,0.037138,0,1,0,1,1,1,0,1,1,1,2,5
2,abdominal,1.46894,1,0,1,0,1,0,1,1,1,0,5,1
3,aortic,1.39638,1,0,1,0,1,0,1,1,1,0,5,1
4,aneurysm,1.31554,1,0,1,0,1,0,1,1,1,0,5,1
6,Haematuria,0.0365789,0,1,0,1,1,1,0,1,1,1,2,5
8,left,-0.909746,0,1,0,1,0,0,0,0,0,1,0,3
9,loin,-0.767129,0,1,0,1,0,0,0,1,0,1,0,4
10,pain,-0.199969,0,1,0,1,1,1,0,1,0,1,1,5
15,abdominal,1.36373,1,0,1,0,1,0,1,1,1,0,5,1
16,aortic,1.33802,1,0,1,0,1,0,1,1,1,0,5,1


In [69]:
# plot_param(np.mean(all_vars['thetas'], 'theta'))

In [70]:
all_vars['thetas']

[array([-1.71125543,  1.51925312,  1.33689954, ...,  0.68681383,
         0.85976038,  0.84679153]),
 array([-1.08283434,  1.24285246, -0.08460055, ...,  1.07695934,
         1.8675878 ,  1.67819513]),
 array([ 1.01123665,  1.26728162,  0.08012674, ..., -0.5812575 ,
         1.46188224,  2.09082617]),
 array([-0.94210427,  2.53939835,  1.05969549, ...,  2.171907  ,
         2.17903668,  1.76749277]),
 array([-1.0637217 ,  2.59402839,  1.50630594, ...,  3.49787901,
         2.05095692,  0.7139552 ]),
 array([-1.01480978,  2.29879911,  1.35700372, ...,  1.90277179,
         2.17865979,  0.93658641]),
 array([-0.19589166,  2.43768913,  1.76178208, ...,  2.50121996,
         1.20404124,  2.32047477]),
 array([-0.10532044,  2.20425723,  1.16382282, ...,  2.2921697 ,
         1.09320269,  2.49965103]),
 array([-0.29027934,  3.14500823,  2.64459694, ...,  2.69039165,
         1.35397061,  2.44263712]),
 array([-0.84778612,  2.16442414,  2.95960066, ...,  2.57208359,
         1.2931394 ,  1.59